In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
pd.options.display.max_rows=200

In [2]:
SEED=411
SPLIT_TEST = False
target = 'deal_probability'

### Basic Agg\Naive Features

In [3]:
## gp features

train = pd.read_csv('data/train.csv', parse_dates=['activation_date'])
train = train.sort_values(['activation_date']).reset_index(drop=True)
test = pd.read_csv('data/test.csv', parse_dates=['activation_date'])
y_train = train.deal_probability.values



'''
gp = pd.read_csv('aggregated_features.csv')
train = train.merge(gp, on='user_id', how='left')
test = test.merge(gp, on='user_id', how='left')
   
agg_cols = list(gp.columns)[1:]
del gp; gc.collect()
'''
agg_cols =[]

In [4]:
# basic features
import string

count = lambda l1,l2: sum([1 for x in l1 if x in l2])
for df in [train, test]:
    df['description'].fillna('unknowndescription', inplace=True)
    df['title'].fillna('unknowntitle', inplace=True)

    #df['day_to_'] = pd.to_datetime(df['activation_date']).dt.day
    df['weekday'] = pd.to_datetime(df['activation_date']).dt.weekday
    
    for col in ['description', 'title']:
        df['num_words_' + col] = df[col].apply(lambda comment: len(comment.split()))
        df['num_unique_words_' + col] = df[col].apply(lambda comment: len(set(w for w in comment.split())))

    df['words_vs_unique_title'] = df['num_unique_words_title'] / df['num_words_title'] * 100
    df['words_vs_unique_description'] = df['num_unique_words_description'] / df['num_words_description'] * 100
    
    df['city'] = df['city'] + ' ' + df['region']
    df['num_desc_punct'] = df['description'].apply(lambda x: count(x, set(string.punctuation)))
    
            
    for col in agg_cols+['price', 'image_top_1']:
         df[col].fillna(-1, inplace=True)
    
    for col in ['param_1', 'param_2', 'param_3']:
         df[col].fillna('unknown'+col, inplace=True)
    
    for col in ['price']:
         df[col].fillna(-1, inplace=True)
            
    
'''
# impute missing values
# Impute image_top_1 with max count
enc = train.groupby('category_name')['image_top_1'].agg(lambda x:x.value_counts().index[0]).astype(np.float32).reset_index()
enc.columns = ['category_name' ,'image_top_1_impute']
train = pd.merge(train, enc, how='left', on='category_name')
test = pd.merge(test, enc, how='left', on='category_name')
train['image_top_1'].fillna(train['image_top_1_impute'], inplace=True)
test['image_top_1'].fillna(test['image_top_1_impute'], inplace=True)
train.drop('image_top_1_impute', axis=1, inplace=True)
test.drop('image_top_1_impute', axis=1, inplace=True)
del enc; gc.collect()

# Impute agg features with med
for f in agg_cols:
    enc = train.groupby('category_name')[f].agg('median').astype(np.float32).reset_index()
    enc.columns = ['category_name' ,f+'_impute']
    train = pd.merge(train, enc, how='left', on='category_name')
    test = pd.merge(test, enc, how='left', on='category_name')
    train[f].fillna(train[f+'_impute'], inplace=True)
    test[f].fillna(test[f+'_impute'], inplace=True)
    train.drop(f+'_impute', axis=1, inplace=True)
    test.drop(f+'_impute', axis=1, inplace=True)
    del enc; gc.collect()
    print(f, ' imputed')
'''

"\n# impute missing values\n# Impute image_top_1 with max count\nenc = train.groupby('category_name')['image_top_1'].agg(lambda x:x.value_counts().index[0]).astype(np.float32).reset_index()\nenc.columns = ['category_name' ,'image_top_1_impute']\ntrain = pd.merge(train, enc, how='left', on='category_name')\ntest = pd.merge(test, enc, how='left', on='category_name')\ntrain['image_top_1'].fillna(train['image_top_1_impute'], inplace=True)\ntest['image_top_1'].fillna(test['image_top_1_impute'], inplace=True)\ntrain.drop('image_top_1_impute', axis=1, inplace=True)\ntest.drop('image_top_1_impute', axis=1, inplace=True)\ndel enc; gc.collect()\n\n# Impute agg features with med\nfor f in agg_cols:\n    enc = train.groupby('category_name')[f].agg('median').astype(np.float32).reset_index()\n    enc.columns = ['category_name' ,f+'_impute']\n    train = pd.merge(train, enc, how='left', on='category_name')\n    test = pd.merge(test, enc, how='left', on='category_name')\n    train[f].fillna(train[f+'_

### Other advanced features

In [5]:
merge_cols = []
merge_cat = []

In [6]:
# vgg 16 pca features
merge = pd.read_pickle('vgg16_image_pca_feature.csv_pkl')
merge_cols.extend(merge.columns.tolist()[1:]) # column 0 is image id

train = train.merge(merge, on='image', how='left')
test = test.merge(merge, on='image', how='left')

del merge; gc.collect()

42

In [7]:
# other pretrained net predictions
files = ['image_top_1_resnet_tsvd.csv_pkl',
         'image_top_1_inception_tsvd.csv_pkl',
         'image_top_1_xception_tsvd.csv_pkl']

for fp in files:
    merge = pd.read_pickle(fp)
    merge_cols.extend(merge.columns.tolist()[1:]) # column 0 is image id

    train = train.merge(merge, on='image', how='left')
    test = test.merge(merge, on='image', how='left')

    del merge; gc.collect()

In [8]:
# opencv features
merge = pd.read_pickle('train_keypoints')
merge_cols.extend(merge.columns.tolist())
train = pd.concat([train, merge], axis=1)
del merge; gc.collect()

merge = pd.read_pickle('test_keypoints')
test = pd.concat([test, merge], axis=1)
del merge; gc.collect()

7

In [9]:
# region macros:
merge = pd.read_csv('region_macro.csv')
merge_cols.extend(merge.columns.tolist()[1:]) # column 0 is region

train = train.merge(merge, on='region', how='left')
test = test.merge(merge, on='region', how='left')

del merge

In [10]:
train.loc[train.image_top_1 != -1,['deal_probability']+merge_cols].corr()

,deal_probability,vgg16_image_pca_feature_1,vgg16_image_pca_feature_2,vgg16_image_pca_feature_3,vgg16_image_pca_feature_4,vgg16_image_pca_feature_5,vgg16_image_pca_feature_6,vgg16_image_pca_feature_7,vgg16_image_pca_feature_8,vgg16_image_pca_feature_9,...,image_top_1_inception_tsvd_1,image_top_1_inception_tsvd_2,image_top_1_xception_tsvd_0,image_top_1_xception_tsvd_1,image_top_1_xception_tsvd_2,corner,keypoint,unemployment_rate,GDP_PC_PPP,HDI
deal_probability,1.000000,0.198504,0.055239,-0.062594,-0.112870,0.024013,0.012015,0.027724,0.016041,-0.025976,...,-0.034509,-0.040940,-0.018907,-0.032871,-0.063092,-0.025657,-0.018589,-0.007327,-0.006204,-0.007730
vgg16_image_pca_feature_1,0.198504,1.000000,-0.000674,-0.001731,0.003262,0.006726,-0.012170,-0.005115,-0.000616,0.000519,...,-0.016344,-0.155211,-0.024092,-0.015289,-0.207773,-0.101334,-0.164998,-0.035786,0.020745,0.027859
vgg16_image_pca_feature_2,0.055239,-0.000674,1.000000,0.004513,-0.002249,-0.004327,-0.008510,-0.003731,-0.007622,0.006898,...,0.174510,-0.054680,0.183707,0.177794,-0.056959,0.049915,0.143314,-0.004640,-0.016885,-0.025344
vgg16_image_pca_feature_3,-0.062594,-0.001731,0.004513,1.000000,-0.004964,0.005331,0.006862,0.006180,0.001889,-0.003499,...,-0.029241,0.033030,-0.009767,-0.032573,0.055859,0.046569,0.067952,0.016858,0.007530,0.002575
vgg16_image_pca_feature_4,-0.112870,0.003262,-0.002249,-0.004964,1.000000,-0.002058,-0.005583,0.000574,-0.003321,0.001678,...,0.084966,-0.014753,0.084109,0.085674,-0.021150,-0.040514,-0.146763,0.004372,-0.006720,0.002337
vgg16_image_pca_feature_5,0.024013,0.006726,-0.004327,0.005331,-0.002058,1.000000,-0.005496,0.000706,-0.017257,0.005988,...,-0.031581,-0.074676,-0.056995,-0.028253,-0.101987,0.161868,0.240648,0.005020,0.004308,0.006604
vgg16_image_pca_feature_6,0.012015,-0.012170,-0.008510,0.006862,-0.005583,-0.005496,1.000000,-0.005300,-0.005890,-0.000090,...,-0.136386,0.164593,-0.164042,-0.142529,0.076480,-0.045213,-0.055713,-0.020014,0.004831,0.009853
vgg16_image_pca_feature_7,0.027724,-0.005115,-0.003731,0.006180,0.000574,0.000706,-0.005300,1.000000,0.000284,0.001419,...,-0.129130,-0.035673,-0.113163,-0.118766,-0.024631,0.266293,0.357230,-0.013813,-0.003976,-0.005617
vgg16_image_pca_feature_8,0.016041,-0.000616,-0.007622,0.001889,-0.003321,-0.017257,-0.005890,0.000284,1.000000,0.008373,...,-0.007587,0.008872,-0.011020,0.001085,0.044610,-0.065216,-0.098503,0.007318,0.007302,0.005305
vgg16_image_pca_feature_9,-0.025976,0.000519,0.006898,-0.003499,0.001678,0.005988,-0.000090,0.001419,0.008373,1.000000,...,0.008726,-0.015901,0.064436,0.020258,-0.047626,-0.111781,-0.140270,-0.006359,0.016775,0.019899


In [11]:
'''
# label_price_features.csv
label_price = pd.read_csv('label_price_features.csv')

train = train.merge(label_price[['item_id', 'price_pred']], on='item_id', how='left')
test = test.merge(label_price[['item_id','price_pred']], on='item_id', how='left')

del label_price; gc.collect()

merge_cols.extend(['price_pred'])
'''

"\n# label_price_features.csv\nlabel_price = pd.read_csv('label_price_features.csv')\n\ntrain = train.merge(label_price[['item_id', 'price_pred']], on='item_id', how='left')\ntest = test.merge(label_price[['item_id','price_pred']], on='item_id', how='left')\n\ndel label_price; gc.collect()\n\nmerge_cols.extend(['price_pred'])\n"

In [12]:
# img confidence score: hurt: high confidence might indicate the wrong catgory...

'''
file_prefix = ['image_top_1_resnet', 'image_top_1_inception', 'image_top_1_xception']
for fp in file_prefix:
    new_col = fp+'_confidence'
    merge_cols.append(new_col)
    
    train_merge = pd.read_pickle(fp+'_train')
    train_merge['image'] = train_merge['image'].map(lambda x: x.replace('.jpg', ''))
    train = train.merge(train_merge[['image', new_col]], on='image', how='left')
    
    test_merge = pd.read_pickle(fp+'_test')
    test_merge['image'] =test_merge['image'].map(lambda x: x.replace('.jpg', ''))
    test = test.merge(test_merge[['image', new_col]], on='image', how='left')
        
    del train_merge, test_merge; gc.collect()
    print(fp, ' processed')
'''

"\nfile_prefix = ['image_top_1_resnet', 'image_top_1_inception', 'image_top_1_xception']\nfor fp in file_prefix:\n    new_col = fp+'_confidence'\n    merge_cols.append(new_col)\n    \n    train_merge = pd.read_pickle(fp+'_train')\n    train_merge['image'] = train_merge['image'].map(lambda x: x.replace('.jpg', ''))\n    train = train.merge(train_merge[['image', new_col]], on='image', how='left')\n    \n    test_merge = pd.read_pickle(fp+'_test')\n    test_merge['image'] =test_merge['image'].map(lambda x: x.replace('.jpg', ''))\n    test = test.merge(test_merge[['image', new_col]], on='image', how='left')\n        \n    del train_merge, test_merge; gc.collect()\n    print(fp, ' processed')\n"

In [13]:
'''
select_cols = ['image_top_1_resnet_confidence', 'image_top_1_inception_confidence', 'image_top_1_xception_confidence']

for df in [train, test]:
    df['image_top_1_resnet_confidence_bin'] = df['image_top_1_resnet_confidence'] > 0.5
    df['image_top_1_inception_confidence_bin'] = df['image_top_1_inception_confidence'] > 0.5
    df['image_top_1_xception_confidence_bin'] = df['image_top_1_xception_confidence'] > 0.5
    df['image_confidence_mean'] = df[select_cols].mean(axis=1)
    df['image_confidence_med'] = df[select_cols].median(axis=1)
    df['image_confidence_max'] = df[select_cols].max(axis=1)
    df['image_confidence_min'] = df[select_cols].min(axis=1)
    df['image_confidence_std'] = df[select_cols].std(axis=1)
    df['image_confidence_vote_count'] = df[['image_top_1_resnet_confidence_bin',
                                            'image_top_1_inception_confidence_bin',
                                            'image_top_1_xception_confidence_bin']].sum(axis=1)
    
merge_cols.extend(['image_top_1_resnet_confidence_bin',
                   'image_top_1_inception_confidence_bin',
                   'image_top_1_xception_confidence_bin',
                   'image_confidence_mean',
                   'image_confidence_med',
                   'image_confidence_max',
                   'image_confidence_min',
                   'image_confidence_std',
                   'image_confidence_vote_count'
                  ])
'''

"\nselect_cols = ['image_top_1_resnet_confidence', 'image_top_1_inception_confidence', 'image_top_1_xception_confidence']\n\nfor df in [train, test]:\n    df['image_top_1_resnet_confidence_bin'] = df['image_top_1_resnet_confidence'] > 0.5\n    df['image_top_1_inception_confidence_bin'] = df['image_top_1_inception_confidence'] > 0.5\n    df['image_top_1_xception_confidence_bin'] = df['image_top_1_xception_confidence'] > 0.5\n    df['image_confidence_mean'] = df[select_cols].mean(axis=1)\n    df['image_confidence_med'] = df[select_cols].median(axis=1)\n    df['image_confidence_max'] = df[select_cols].max(axis=1)\n    df['image_confidence_min'] = df[select_cols].min(axis=1)\n    df['image_confidence_std'] = df[select_cols].std(axis=1)\n    df['image_confidence_vote_count'] = df[['image_top_1_resnet_confidence_bin',\n                                            'image_top_1_inception_confidence_bin',\n                                            'image_top_1_xception_confidence_bin']].sum(ax

In [14]:
# geo features: https://www.kaggle.com/frankherfert/region-and-city-details-with-lat-lon-and-clusters/notebook

city_region_features = pd.read_csv('avito_region_city_features.csv')

train.loc[:, 'city_region'] = train['city']# + ' ' + train['region']
test.loc[:, 'city_region'] = test['city']# + ' ' + test['region']

cols = ['latitude', 'longitude', 'lat_lon_hdbscan_cluster_05_03', 'lat_lon_hdbscan_cluster_10_03', 'lat_lon_hdbscan_cluster_20_03']
train = train.merge(city_region_features[cols+['city_region']], on='city_region', how='left')
test = test.merge(city_region_features[cols+['city_region']], on='city_region', how='left')

train.drop('city_region', axis=1, inplace=True)
test.drop('city_region', axis=1, inplace=True)
del city_region_features; gc.collect()

186

In [15]:
merge_cols.extend(cols)
merge_cat.extend(['lat_lon_hdbscan_cluster_05_03', 'lat_lon_hdbscan_cluster_10_03', 'lat_lon_hdbscan_cluster_20_03'])

In [16]:
# fm features: self train for feature interaction

file_prefix = ['cityxcatxusertype', 'imgxcityxcat', 'imgxisqnxusertype']
for fp in file_prefix:
    train_merge = pd.read_csv('train_'+fp+'_features.csv')
    for col in train_merge.columns:
        train.loc[:, fp+col] = train_merge[col]
        merge_cols.append(fp+col)
        
    test_merge = pd.read_csv('test_'+fp+'_features.csv')
    for col in test_merge.columns:
        test.loc[:, fp+col] = test_merge[col]
        
    del train_merge, test_merge; gc.collect()
    print(fp, ' processed')
    
# remove less significant features:
for f in['imgxisqnxusertypeuser_type_fm_factor_1', 'imgxisqnxusertypeuser_type_fm_bias']:
    train.drop(f, axis=1, inplace=True)
    test.drop(f, axis=1, inplace=True)
    merge_cols.remove(f)

cityxcatxusertype  processed
imgxcityxcat  processed
imgxisqnxusertype  processed


In [17]:
## img meta

def clean_img(img):
    return img.split('.')[0]

img_size = pd.read_pickle('image_size')
img_size.columns = ['image', 'img_size']
img_size['image'] = img_size['image'].apply(clean_img)

img_info = pd.read_pickle('ImageInfo')
img_info['image'] = img_info['image'].apply(clean_img)

train = train.merge(img_size, how='left', on='image')
train = train.merge(img_info, how='left', on='image')
test = test.merge(img_size, how='left', on='image')
test = test.merge(img_info, how='left', on='image')

merge_cols.extend(img_size.columns[img_size.columns != 'image'].tolist())
merge_cols.extend(img_info.columns[img_info.columns != 'image'].tolist())
   
del img_size, img_info; gc.collect()

146

In [18]:
## nima scores

file_prefix = ['nasnet_nima_']
for fp in file_prefix:
    train_merge = pd.read_csv('data/train_'+fp+'features_ver2.csv')
    print(train_merge.shape)
    for col in train_merge.columns:
        if col == 'image':
            continue
        train.loc[:, fp+col] = train_merge[col]
        merge_cols.append(fp+col)
        
    test_merge = pd.read_csv('data/test_'+fp+'features_ver2.csv')
    for col in test_merge.columns:
        if col == 'image':
            continue
        test.loc[:, fp+col] = test_merge[col]
        
    del train_merge, test_merge; gc.collect()
    print(fp, ' processed')

# remove nasnet_nima_mean
'''
for f in ['nasnet_nima_mean']:
    train.drop(f, axis=1, inplace=True)
    test.drop(f, axis=1, inplace=True)
    merge_cols.remove(f)
'''

(1503424, 11)
nasnet_nima_  processed


"\nfor f in ['nasnet_nima_mean']:\n    train.drop(f, axis=1, inplace=True)\n    test.drop(f, axis=1, inplace=True)\n    merge_cols.remove(f)\n"

In [19]:
# active features

active_features = ['active_price_pred', 'active_duration_pred', 'active_ad_count_pred']

train_merge = pd.read_csv('train_active_features_concated.csv')
print(train_merge.shape)
for col in active_features:
    train.loc[:, col] = train_merge[col]
    merge_cols.append(col)

test_merge = pd.read_csv('test_active_features_concated.csv')
for col in active_features:
    test.loc[:, col] = test_merge[col]

del train_merge, test_merge; gc.collect()

(1503424, 3)


97

In [20]:
# zijun top features
'''
top_features = [
    'price_diff_1',
    'region_ME',
    'score_diff_4',
    'user_type_pcate_cate_cumcount',
    'title_num_chars',
    'region_day_count',
    'MORECOUNT_2',
    'seq_region_count',
    'user_id_city_region_cumcount',
    'price_diff_5',
    'price_diff_3',
    'seq_city_count']

train_merge = pd.read_pickle('zijun_top50_train.pkl.gz')
print(train_merge.shape)
for col in top_features:
    train.loc[:, col] = train_merge[col].values
    merge_cols.append(col)

test_merge = pd.read_pickle('zijun_top50_test.pkl.gz')
for col in top_features:
    test.loc[:, col] = test_merge[col].values

del train_merge, test_merge; gc.collect()
'''

"\ntop_features = [\n    'price_diff_1',\n    'region_ME',\n    'score_diff_4',\n    'user_type_pcate_cate_cumcount',\n    'title_num_chars',\n    'region_day_count',\n    'MORECOUNT_2',\n    'seq_region_count',\n    'user_id_city_region_cumcount',\n    'price_diff_5',\n    'price_diff_3',\n    'seq_city_count']\n\ntrain_merge = pd.read_pickle('zijun_top50_train.pkl.gz')\nprint(train_merge.shape)\nfor col in top_features:\n    train.loc[:, col] = train_merge[col].values\n    merge_cols.append(col)\n\ntest_merge = pd.read_pickle('zijun_top50_test.pkl.gz')\nfor col in top_features:\n    test.loc[:, col] = test_merge[col].values\n\ndel train_merge, test_merge; gc.collect()\n"

In [21]:
# split local test data after merge
if SPLIT_TEST:
    del test; gc.collect()
    
    import random
    test_ix = random.sample(list(np.arange(train.shape[0])), 375000)
    test = train.loc[test_ix,:].reset_index(drop=True)
    y_test = y_train[test_ix]
    
    new_train_filter = ~train.index.isin(test_ix)
    train = train.loc[new_train_filter,:].reset_index(drop=True)
    y_train = y_train[new_train_filter]

In [22]:
if merge_cols != []:
    for df in [train, test]:
        print(df.loc[:, merge_cols].isnull().sum())
        df.loc[:, merge_cols] = df[merge_cols].fillna(-1.)

vgg16_image_pca_feature_1                       112588
vgg16_image_pca_feature_2                       112588
vgg16_image_pca_feature_3                       112588
vgg16_image_pca_feature_4                       112588
vgg16_image_pca_feature_5                       112588
vgg16_image_pca_feature_6                       112588
vgg16_image_pca_feature_7                       112588
vgg16_image_pca_feature_8                       112588
vgg16_image_pca_feature_9                       112588
vgg16_image_pca_feature_10                      112588
vgg16_image_pca_feature_11                      112588
vgg16_image_pca_feature_12                      112588
vgg16_image_pca_feature_13                      112588
vgg16_image_pca_feature_14                      112588
vgg16_image_pca_feature_15                      112588
vgg16_image_pca_feature_16                      112588
vgg16_image_pca_feature_17                      112588
vgg16_image_pca_feature_18                      112588
vgg16_imag

In [23]:
target = 'deal_probability'
predictors = [
    'num_desc_punct', 
    'words_vs_unique_description', 'num_unique_words_description', 'num_unique_words_title', 'num_words_description', 'num_words_title',
    'avg_times_up_user', 'avg_days_up_user', 'med_times_up_user', 'med_days_up_user', 'n_user_items', 
    'price', 'item_seq_number'
] + merge_cols

# , 'user_type',
categorical = [
    'image_top_1', 'param_1', 'param_2', 'param_3', 
    'city', 'region', 'category_name', 'parent_category_name', 'user_type',
] 

predictors = predictors + categorical
categorical = categorical + merge_cat

In [24]:
predictors, categorical

(['num_desc_punct',
  'words_vs_unique_description',
  'num_unique_words_description',
  'num_unique_words_title',
  'num_words_description',
  'num_words_title',
  'avg_times_up_user',
  'avg_days_up_user',
  'med_times_up_user',
  'med_days_up_user',
  'n_user_items',
  'price',
  'item_seq_number',
  'vgg16_image_pca_feature_1',
  'vgg16_image_pca_feature_2',
  'vgg16_image_pca_feature_3',
  'vgg16_image_pca_feature_4',
  'vgg16_image_pca_feature_5',
  'vgg16_image_pca_feature_6',
  'vgg16_image_pca_feature_7',
  'vgg16_image_pca_feature_8',
  'vgg16_image_pca_feature_9',
  'vgg16_image_pca_feature_10',
  'vgg16_image_pca_feature_11',
  'vgg16_image_pca_feature_12',
  'vgg16_image_pca_feature_13',
  'vgg16_image_pca_feature_14',
  'vgg16_image_pca_feature_15',
  'vgg16_image_pca_feature_16',
  'vgg16_image_pca_feature_17',
  'vgg16_image_pca_feature_18',
  'vgg16_image_pca_feature_19',
  'vgg16_image_pca_feature_20',
  'vgg16_image_pca_feature_21',
  'vgg16_image_pca_feature_22',
  

### More feature engineering

In [25]:
# join params
'''
for df in [train, test]:
    df['param_join'] = df['param_1'].astype(str) + ' ' + df['param_2'].astype(str) + ' ' + df['param_3'].astype(str)
                                                                                                         
categorical.append('param_join') 
predictors.append('param_join')
'''
#remove original param
#
#for cat in ['param_1', 'param_2', 'param_3']:
#    categorical.remove(cat) 
#    predictors.remove(cat)
#

"\nfor df in [train, test]:\n    df['param_join'] = df['param_1'].astype(str) + ' ' + df['param_2'].astype(str) + ' ' + df['param_3'].astype(str)\n                                                                                                         \ncategorical.append('param_join') \npredictors.append('param_join')\n"

In [26]:
#user features
'''
from copy import deepcopy as cp
col = 'user_id'
user_vc = train[col].append(test[col]).value_counts()
old_user_vc = cp(user_vc)
user_vc = user_vc[user_vc>30]
users = user_vc.index
user_filter = users.isin(train[col]) & users.isin(test[col])

print('Original feature types # = {}, new feature types # = {}'.format(
        len(old_user_vc), pd.Series(user_filter).value_counts()[1]))

print(train.loc[train[col].isin(users[user_filter]), col].value_counts().sort_index())
print(test.loc[test[col].isin(users[user_filter]), col].value_counts().sort_index())
print(train.loc[train[col].isin(users[user_filter]), :].shape[0]/train.shape[0]*100.)
print(test.loc[test[col].isin(users[user_filter]), :].shape[0]/test.shape[0]*100.)

train.loc[~train[col].isin(users[user_filter]), col] = 'others_users'
test.loc[~test[col].isin(users[user_filter]), col] = 'others_users'

gp = train.groupby('user_id')

for col in ['item_seq_number']: # 'price', , 'num_desc_punct'
    mean_mapping, std_mapping = gp[col].mean(), gp[col].std()
    
    train.loc[:, col+'_user_mean'] = train['user_id'].map(mean_mapping).fillna(0.)
    test.loc[:, col+'_user_mean'] = test['user_id'].map(mean_mapping).fillna(0.)
    
    train.loc[:, col+'_user_std'] = train['user_id'].map(std_mapping).fillna(0.)
    test.loc[:, col+'_user_std'] = test['user_id'].map(std_mapping).fillna(0.)
    
    predictors.extend([col+'_user_mean', col+'_user_std'])
    print(col+' processed')
'''

"\nfrom copy import deepcopy as cp\ncol = 'user_id'\nuser_vc = train[col].append(test[col]).value_counts()\nold_user_vc = cp(user_vc)\nuser_vc = user_vc[user_vc>30]\nusers = user_vc.index\nuser_filter = users.isin(train[col]) & users.isin(test[col])\n\nprint('Original feature types # = {}, new feature types # = {}'.format(\n        len(old_user_vc), pd.Series(user_filter).value_counts()[1]))\n\nprint(train.loc[train[col].isin(users[user_filter]), col].value_counts().sort_index())\nprint(test.loc[test[col].isin(users[user_filter]), col].value_counts().sort_index())\nprint(train.loc[train[col].isin(users[user_filter]), :].shape[0]/train.shape[0]*100.)\nprint(test.loc[test[col].isin(users[user_filter]), :].shape[0]/test.shape[0]*100.)\n\ntrain.loc[~train[col].isin(users[user_filter]), col] = 'others_users'\ntest.loc[~test[col].isin(users[user_filter]), col] = 'others_users'\n\ngp = train.groupby('user_id')\n\nfor col in ['item_seq_number']: # 'price', , 'num_desc_punct'\n    mean_mapping,

In [27]:
partial = pd.concat([train[categorical+['price', 'item_seq_number']],
                    test[categorical+['price', 'item_seq_number']]]).reset_index(drop=True)

In [28]:
# price and item_seq_number enc
'''
for cat in ['image_top_1', 'city', 'param_join', 'category_name']:
    gp = partial.groupby(cat)
  
    for df in [train, test]:
        #df.loc[:, cat+'_price_avg'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].mean())).fillna(-1.)
        #df.loc[:, cat+'_price_med'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].median())).fillna(-1.)
        #df.loc[:, cat+'_price_sum'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].sum())).fillna(-1.)
        #df.loc[:, cat+'_price_std'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].std())).fillna(-1.)
        
        df.loc[:, cat+'_price_na_count'] = df[cat].map(gp['price'].apply(lambda x: x[x<0].count())).fillna(-1.)
        df.loc[:, cat+'_item_seq_num_avg'] = df[cat].map(gp['item_seq_number'].mean()).fillna(-1.)
        df.loc[:, cat+'_item_seq_num_med'] = df[cat].map(gp['item_seq_number'].median()).fillna(-1.)
        df.loc[:, cat+'_item_seq_num_sum'] = df[cat].map(gp['item_seq_number'].sum()).fillna(-1.)
        df.loc[:, cat+'_item_seq_num_unique_count'] = \
                                            df[cat].map(gp['item_seq_number'].apply(lambda x: pd.Series(x).nunique())).fillna(-1.)
        df.loc[:, cat+'_item_seq_num_std'] = df[cat].map(gp['item_seq_number'].std()).fillna(-1.)

    predictors = predictors + \
        [cat+'_price_na_count',
         cat+'_item_seq_num_avg',
         cat+'_item_seq_num_sum',
         cat+'_item_seq_num_unique_count',
         cat+'_item_seq_num_std']
        
    del gp; gc.collect()
    print(cat, ' processed')
'''

"\nfor cat in ['image_top_1', 'city', 'param_join', 'category_name']:\n    gp = partial.groupby(cat)\n  \n    for df in [train, test]:\n        #df.loc[:, cat+'_price_avg'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].mean())).fillna(-1.)\n        #df.loc[:, cat+'_price_med'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].median())).fillna(-1.)\n        #df.loc[:, cat+'_price_sum'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].sum())).fillna(-1.)\n        #df.loc[:, cat+'_price_std'] = df[cat].map(gp['price'].apply(lambda x: x[x>=0].std())).fillna(-1.)\n        \n        df.loc[:, cat+'_price_na_count'] = df[cat].map(gp['price'].apply(lambda x: x[x<0].count())).fillna(-1.)\n        df.loc[:, cat+'_item_seq_num_avg'] = df[cat].map(gp['item_seq_number'].mean()).fillna(-1.)\n        df.loc[:, cat+'_item_seq_num_med'] = df[cat].map(gp['item_seq_number'].median()).fillna(-1.)\n        df.loc[:, cat+'_item_seq_num_sum'] = df[cat].map(gp['item_seq_number'].sum()).fillna(-1.)\n    

In [29]:
'''
# binned price

sorted_prices = sorted(partial.price.values)
_, bins = pd.qcut(sorted_prices, 80, labels=None, retbins=True, precision=3, duplicates='drop')
partial['price_binned'] = \
    pd.cut(partial.price, bins=bins, labels=np.arange(len(bins)-1), retbins=False)
partial['price_binned'] = partial['price_binned'].astype(int)
print(partial['price_binned'].value_counts())

train.loc[:, 'price_binned'] = partial['price_binned'].values[:train.shape[0]]
test.loc[:, 'price_binned'] = partial['price_binned'].values[train.shape[0]:]

categorical.append('price_binned')
predictors.append('price_binned')
'''

"\n# binned price\n\nsorted_prices = sorted(partial.price.values)\n_, bins = pd.qcut(sorted_prices, 80, labels=None, retbins=True, precision=3, duplicates='drop')\npartial['price_binned'] =     pd.cut(partial.price, bins=bins, labels=np.arange(len(bins)-1), retbins=False)\npartial['price_binned'] = partial['price_binned'].astype(int)\nprint(partial['price_binned'].value_counts())\n\ntrain.loc[:, 'price_binned'] = partial['price_binned'].values[:train.shape[0]]\ntest.loc[:, 'price_binned'] = partial['price_binned'].values[train.shape[0]:]\n\ncategorical.append('price_binned')\npredictors.append('price_binned')\n"

In [30]:
# binned isqn

'''
sorted_isqn = sorted(partial.item_seq_number.values)

_, bins = pd.qcut(sorted_isqn, 1000, labels=None, retbins=True, precision=3, duplicates='drop')
partial['isqn_binned'] = \
    pd.cut(partial.item_seq_number.values, bins=bins, labels=np.arange(len(bins)-1), retbins=False)
partial['isqn_binned'] = partial['isqn_binned'].astype(int)
print(partial['isqn_binned'].value_counts())

train.loc[:, 'isqn_binned'] = partial['isqn_binned'].values[:train.shape[0]]
test.loc[:, 'isqn_binned'] = partial['isqn_binned'].values[train.shape[0]:]

categorical.append('isqn_binned')
predictors.append('isqn_binned')
'''

"\nsorted_isqn = sorted(partial.item_seq_number.values)\n\n_, bins = pd.qcut(sorted_isqn, 1000, labels=None, retbins=True, precision=3, duplicates='drop')\npartial['isqn_binned'] =     pd.cut(partial.item_seq_number.values, bins=bins, labels=np.arange(len(bins)-1), retbins=False)\npartial['isqn_binned'] = partial['isqn_binned'].astype(int)\nprint(partial['isqn_binned'].value_counts())\n\ntrain.loc[:, 'isqn_binned'] = partial['isqn_binned'].values[:train.shape[0]]\ntest.loc[:, 'isqn_binned'] = partial['isqn_binned'].values[train.shape[0]:]\n\ncategorical.append('isqn_binned')\npredictors.append('isqn_binned')\n"

In [31]:
def add_noise(series, noise_level): 
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
                      
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [32]:
# cat encoding

'''
#label encode
for feature in categorical:
    print(f'Transforming {feature}...')
    encoder = LabelEncoder()
    encoder.fit(train[feature].append(test[feature]).astype(str))
    
    train[feature] = encoder.transform(train[feature].astype(str))
    test[feature] = encoder.transform(test[feature].astype(str))

#cnt encode
for feature in categorical:
    if feature == 'user_type':
        continue
        
    gp = train.groupby(feature)[feature].count()
    train.loc[:, feature+'_cnt'] = train[feature].map(gp)
    test.loc[:, feature+'_cnt'] = test[feature].map(gp)
    predictors.append(feature+'_cnt')
    
    del gp; gc.collect()


#target encode
from sklearn.model_selection import KFold
from scipy.stats import kurtosis
fold_num = 5
kf = KFold(fold_num, shuffle=True, random_state=SEED)

for col in categorical+['user_id']:    
    
    #train.loc[:, col+'_dp_mean_enc'] = np.zeros((train.shape[0],))
    #train.loc[:, col+'_dp_std_enc'] = np.zeros((train.shape[0],))
    #test.loc[:, col+'_dp_mean_enc'] = np.zeros((test.shape[0],))
    #test.loc[:, col+'_dp_std_enc'] = np.zeros((test.shape[0],))
    ##test.loc[:, col+'_dp_kurtosis_enc'] = np.zeros((test.shape[0],))
    #
    #for train_ix, val_ix in kf.split(train):
    #    tr_X = train.loc[train_ix, :]
    #
    #    gp = tr_X.groupby(col)['deal_probability']
    #    mapping, mapping_std = gp.mean(), gp.std()
    #    mapping_kurtosis = gp.apply(lambda x: kurtosis(x, nan_policy='omit'))
    #    
    #    train.loc[val_ix, col+'_dp_mean_enc'] = train.loc[val_ix, col].map(mapping).fillna(-1.)
    #    train.loc[val_ix, col+'_dp_std_enc'] = train.loc[val_ix, col].map(mapping_std).fillna(-1.)
    #    #train.loc[val_ix, col+'_dp_kurtosis_enc'] = train.loc[val_ix, col].map(mapping_kurtosis).fillna(-1.)
    #    
    #    test.loc[:, col+'_dp_mean_enc'] += test.loc[:, col].map(mapping).fillna(-1.)
    #    test.loc[:, col+'_dp_std_enc'] += test.loc[:, col].map(mapping_std).fillna(-1.)
    #    #test.loc[:, col+'_dp_kurtosis_enc'] += test.loc[:, col].map(mapping_kurtosis).fillna(-1.)
    # 
    #    del gp, mapping, mapping_std, mapping_kurtosis; gc.collect()
        
    #test.loc[:, col+'_dp_mean_enc'] /= fold_num
    #test.loc[:, col+'_dp_std_enc'] /= fold_num
    #print(col + ' processed.')
    #predictors.extend([col+'_dp_mean_enc', col+'_dp_std_enc'])

    trn, sub = target_encode(train[col], 
                             test[col], 
                             target=pd.Series(data=y_train, name='deal_probability'),  
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
                            
    train.loc[:, col+'_dp_mean_enc'] = trn.astype('float32')
    test.loc[:, col+'_dp_mean_enc'] = sub.astype('float32')
    predictors.extend([col+'_dp_mean_enc'])  
'''

"\n#label encode\nfor feature in categorical:\n    print(f'Transforming {feature}...')\n    encoder = LabelEncoder()\n    encoder.fit(train[feature].append(test[feature]).astype(str))\n    \n    train[feature] = encoder.transform(train[feature].astype(str))\n    test[feature] = encoder.transform(test[feature].astype(str))\n\n#cnt encode\nfor feature in categorical:\n    if feature == 'user_type':\n        continue\n        \n    gp = train.groupby(feature)[feature].count()\n    train.loc[:, feature+'_cnt'] = train[feature].map(gp)\n    test.loc[:, feature+'_cnt'] = test[feature].map(gp)\n    predictors.append(feature+'_cnt')\n    \n    del gp; gc.collect()\n\n\n#target encode\nfrom sklearn.model_selection import KFold\nfrom scipy.stats import kurtosis\nfold_num = 5\nkf = KFold(fold_num, shuffle=True, random_state=SEED)\n\nfor col in categorical+['user_id']:    \n    \n    #train.loc[:, col+'_dp_mean_enc'] = np.zeros((train.shape[0],))\n    #train.loc[:, col+'_dp_std_enc'] = np.zeros((t

In [33]:
# remove all categorical variables
'''
predictors = [f for f in predictors if f not in mean_enc_cols]
categorical = []
'''

'\npredictors = [f for f in predictors if f not in mean_enc_cols]\ncategorical = []\n'

In [34]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else: df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [35]:
train.drop(['activation_date', 'item_id', 'user_id', 'title', 'description'], axis=1, inplace=True)
test.drop(['activation_date', 'item_id', 'user_id', 'title', 'description'], axis=1, inplace=True)

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 1284.66 MB
Memory usage after optimization is: 429.30 MB
Decreased by 66.6%
Memory usage of dataframe is 430.58 MB
Memory usage after optimization is: 150.76 MB
Decreased by 65.0%


In [36]:
predictors, categorical, len(predictors)

(['num_desc_punct',
  'words_vs_unique_description',
  'num_unique_words_description',
  'num_unique_words_title',
  'num_words_description',
  'num_words_title',
  'avg_times_up_user',
  'avg_days_up_user',
  'med_times_up_user',
  'med_days_up_user',
  'n_user_items',
  'price',
  'item_seq_number',
  'vgg16_image_pca_feature_1',
  'vgg16_image_pca_feature_2',
  'vgg16_image_pca_feature_3',
  'vgg16_image_pca_feature_4',
  'vgg16_image_pca_feature_5',
  'vgg16_image_pca_feature_6',
  'vgg16_image_pca_feature_7',
  'vgg16_image_pca_feature_8',
  'vgg16_image_pca_feature_9',
  'vgg16_image_pca_feature_10',
  'vgg16_image_pca_feature_11',
  'vgg16_image_pca_feature_12',
  'vgg16_image_pca_feature_13',
  'vgg16_image_pca_feature_14',
  'vgg16_image_pca_feature_15',
  'vgg16_image_pca_feature_16',
  'vgg16_image_pca_feature_17',
  'vgg16_image_pca_feature_18',
  'vgg16_image_pca_feature_19',
  'vgg16_image_pca_feature_20',
  'vgg16_image_pca_feature_21',
  'vgg16_image_pca_feature_22',
  

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
SEED=411 
kf = KFold(5, shuffle=True, random_state=SEED)
%matplotlib inline

In [38]:
'''
remove_cols = [
    'cityxcatxusertypecategory_name_fm_factor_0',
    'cityxcatxusertypecategory_name_fm_factor_1',
    'cityxcatxusertypecategory_name_fm_bias',
    'cityxcatxusertypeuser_type_fm_bias',
    'imgxcityxcatcategory_name_fm_factor_0',
    'imgxcityxcatcategory_name_fm_factor_1',
    'imgxisqnxusertypeuser_type_fm_factor_0', 
    'user_type',
    'image_top_1_item_seq_num_unique_count',
    'category_name_price_na_count',
    'category_name_item_seq_num_sum', 'category_name_item_seq_num_unique_count',
    'image_top_1_cnt', 'param_2_cnt', 'param_3_cnt', 'category_name_cnt',
    'parent_category_name_cnt', 'param_join_cnt']

for col in remove_cols:
    if col in predictors:
        predictors.remove(col)
    if col in categorical:
        categorical.remove(col)
    if col in train.columns.tolist():
        train.drop(col, axis=1, inplace=True)
        test.drop(col, axis=1, inplace=True)
'''

"\nremove_cols = [\n    'cityxcatxusertypecategory_name_fm_factor_0',\n    'cityxcatxusertypecategory_name_fm_factor_1',\n    'cityxcatxusertypecategory_name_fm_bias',\n    'cityxcatxusertypeuser_type_fm_bias',\n    'imgxcityxcatcategory_name_fm_factor_0',\n    'imgxcityxcatcategory_name_fm_factor_1',\n    'imgxisqnxusertypeuser_type_fm_factor_0', \n    'user_type',\n    'image_top_1_item_seq_num_unique_count',\n    'category_name_price_na_count',\n    'category_name_item_seq_num_sum', 'category_name_item_seq_num_unique_count',\n    'image_top_1_cnt', 'param_2_cnt', 'param_3_cnt', 'category_name_cnt',\n    'parent_category_name_cnt', 'param_join_cnt']\n\nfor col in remove_cols:\n    if col in predictors:\n        predictors.remove(col)\n    if col in categorical:\n        categorical.remove(col)\n    if col in train.columns.tolist():\n        train.drop(col, axis=1, inplace=True)\n        test.drop(col, axis=1, inplace=True)\n"

### Modeling: Local test or dump features for future other learnings

In [39]:
if SPLIT_TEST is True:
    import pickle
    train_text = pickle.load(open('train_text_dense_new', 'rb'))
    
    new_test = np.hstack([train_text[test_ix,:], test[predictors].values])
    new_train = np.hstack([train_text[new_train_filter,:], train[predictors].values])
    
    text_feature_names = ['text_'+str(i) for i in range(train_text.shape[1])]
    predictors = np.hstack([
        text_feature_names,
        predictors,
    ])
    print('predictors updated!')
    
    del train_text, train, test; gc.collect()
    train, test = new_train, new_test

In [40]:
def feature_select(rounds):
    global predictors, categorical
    
    for i in range(rounds):
        rounds = 2000
        early_stop_rounds = 100
        trial_seed = 0

        if SPLIT_TEST is True:
            test_pred = np.zeros((test.shape[0],))
            
        feature_importances = None
        for train_ix, val_ix in kf.split(train):
            trial_seed += 1
            params = {
                'objective' : 'regression',
                'metric' : 'rmse',
                'num_leaves' : 31,
                'max_depth': 5,
                'learning_rate' : 0.1,
                'bagging_fraction': 1.,
                'bagging_freq': 1,    
                'feature_fraction' : .6,
                'feature_fraction_seed': trial_seed,
                'reg_alpha': 0.0, 
                'reg_lambda': 0.0,
                'verbosity' : -1,
                'n_jobs': 4
            }

            dtrain = lgb.Dataset(train[train_ix, :], label=y_train[train_ix],
                                 feature_name=list(predictors), 
                                 categorical_feature=categorical)
            dvalid = lgb.Dataset(train[val_ix, :], label=y_train[val_ix],
                                 feature_name=list(predictors), 
                                 categorical_feature=categorical)

            model = lgb.train(params, dtrain, 
                              valid_sets=[dtrain, dvalid], 
                              valid_names=['train', 'valid'],
                              num_boost_round=rounds, 
                              early_stopping_rounds=early_stop_rounds, 
                              verbose_eval=100)
            
            test_pred += model.predict(test)
            
            current_feat_imp = model.feature_importance(importance_type='gain')
            pd.Series(current_feat_imp).hist()
            plt.show()

            if feature_importances is None:
                feature_importances = current_feat_imp
            else:
                feature_importances += current_feat_imp

            del dtrain, dvalid; gc.collect()
        
        if SPLIT_TEST is True:
            test_pred /= 5
            test_pred = np.clip(test_pred, 0., 1.)
            print('test loss=', ((y_test-test_pred)**2.).mean()**.5)

        th = 10 # unit: percent
        th = np.percentile(feature_importances, th)
        print('threshold=', th)
        print('Original dense features # =', len(feature_importances))
        print('Removed feature # =', (feature_importances<=th).astype(int).sum())
        print('Removed feature importances =', feature_importances[feature_importances<=th])
        print('Removed features:\n', np.array(predictors)[feature_importances<=th])
    
        df = pd.DataFrame()
        df['feature_name'] = predictors
        df['feature_importances'] = feature_importances
        print(df.sort_values('feature_importances', ascending=False).reset_index(drop=True))
        
        predictors = list(predictors)
        for f in np.array(predictors)[feature_importances<=th]:
            if f in categorical:
                categorical.remove(f)
            if f in predictors:
                predictors.remove(f)

In [41]:
print(len(merge_cols))
merge_cols

105


['vgg16_image_pca_feature_1',
 'vgg16_image_pca_feature_2',
 'vgg16_image_pca_feature_3',
 'vgg16_image_pca_feature_4',
 'vgg16_image_pca_feature_5',
 'vgg16_image_pca_feature_6',
 'vgg16_image_pca_feature_7',
 'vgg16_image_pca_feature_8',
 'vgg16_image_pca_feature_9',
 'vgg16_image_pca_feature_10',
 'vgg16_image_pca_feature_11',
 'vgg16_image_pca_feature_12',
 'vgg16_image_pca_feature_13',
 'vgg16_image_pca_feature_14',
 'vgg16_image_pca_feature_15',
 'vgg16_image_pca_feature_16',
 'vgg16_image_pca_feature_17',
 'vgg16_image_pca_feature_18',
 'vgg16_image_pca_feature_19',
 'vgg16_image_pca_feature_20',
 'vgg16_image_pca_feature_21',
 'vgg16_image_pca_feature_22',
 'vgg16_image_pca_feature_23',
 'vgg16_image_pca_feature_24',
 'vgg16_image_pca_feature_25',
 'vgg16_image_pca_feature_26',
 'vgg16_image_pca_feature_27',
 'vgg16_image_pca_feature_28',
 'vgg16_image_pca_feature_29',
 'vgg16_image_pca_feature_30',
 'image_top_1_resnet_tsvd_0',
 'image_top_1_resnet_tsvd_1',
 'image_top_1_resne

In [42]:
# dump features only if not in test mode
if SPLIT_TEST is False:
    import pickle
    pickle.dump(train[merge_cols], open('train_selected_features', 'wb'))
    pickle.dump(test[merge_cols], open('test_selected_features', 'wb'))
    #pickle.dump(categorical, open('selected_cat_feature_names', 'wb'))
else:
    feature_select(1)
    print(predictors)
    print(categorical)

#### Feature Selection Local CV Records (Simple lgb trees)
* baseline = 0.221558
* +last all (smoothing mean enc) = cv: 0.22127, local test: 0.2209x
* +last all (20 fold oof mean enc) = cv: 0.219657, local test: 0.2200x
* +last all (20 fold oof mean enc) + geo = cv: 0.219868, local test: 0.21955
* +impute image top 1 and agg features + label price feature = 0.21888







#### Old experiments
* +user feature => .
* +joined param123 => .
* +price&item encode =>.
* price(na_count only)&item encode => .
* user_type cnt (city+param_join only)&price(na_count only)&item(exclude parant_cat, param_3, user_type) encode => .
* remove user_type count, remove user mean\std, + binned_price => bad a lot, cut!
* remove user_type count, remove user mean\std, + binned_isqn => bad a lot, cut!
* remove user_type count, remove user mean\std, + fm => .
* remove user_type count, use price_na+isqn encode with only (city, category_name, img_top_1, param_join), remove useless fm features => .
* +img meta =>.
* +nima => .
* +nima (nima mean removed) => .
* +active features => .
* +zijun top features => .
* +kurtosis encode => .